# Nine men's morris GUI

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
import ipycanvas
import math

In [ ]:
import logging

# create logger with ''
logger = logging.getLogger('GUI')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('log.txt')
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)

## Konstanten

(TODO: Konstanten erklären und Schema erstellen)

In [ ]:
# Board constants
BOARD_SIZE = 500
CANVAS_PADDING = 30
ROW_WIDTH = 60
PLAYER_PIECE_RADIUS = 12
DEFAULT_PIECE_RADIUS = 5

DEFAULT_STONE_LINE_WIDTH = 1
SELECTED_STONE_LINE_WIDTH = 3

STASH_STONES_SPACING = 5

STASH_HEIGHT = ((PLAYER_PIECE_RADIUS * 2 ) * 2) + (STASH_STONES_SPACING) + (CANVAS_PADDING * 2)
STASH_WIDTH = ((PLAYER_PIECE_RADIUS * 2 ) * 9) + (STASH_STONES_SPACING * 8) + (CANVAS_PADDING * 2)

CANVAS_HEIGHT = BOARD_SIZE
CANVAS_WIDTH = 1500

STASH_STARTING_POINT_X = BOARD_SIZE
STASH_STARTING_POINT_Y = CANVAS_HEIGHT - STASH_HEIGHT

# text
TEXT_X = BOARD_SIZE + CANVAS_PADDING
TEXT_Y = CANVAS_PADDING
TEXT_MAX_WIDTH = CANVAS_WIDTH - BOARD_SIZE - 2 * CANVAS_PADDING
TEXT_FONT = '14px sans-serif'
TEXT_VERTICAL_PADDING = 40


# players
NO_PLAYER = ' '
PLAYER_1 = 'w'
PLAYER_2 = 'b'


# color constants
CANVAS_BACKGROUND_COLOR = '#ffffff'
BOARD_FOREGROUND_COLOR = '#191919'
BOARD_BACKGROUND_COLOR = '#ffffcb'
STASH_BACKGROUND_COLOR = '#dedede'
PLAYER_1_FILL_COLOR = '#E8E8E8'
PLAYER_1_STROKE_COLOR = '#191919'
PLAYER_2_FILL_COLOR = '#3D3D3D'
PLAYER_2_STROKE_COLOR = '#191919'
TEXT_COLOR = '#000000'

# enabled shadow constants
SHADOW_COLOR_ENABLED = '#000000'
SHADOW_OFFSET_X_ENABLED = 2
SHADOW_OFFSET_Y_ENABLED = 2
SHADOW_BLUR_ENABLED = 2

# disabled shadow constants
SHADOW_COLOR_DISABLED = 'rgba(0, 0, 0, 0)'
SHADOW_OFFSET_X_DISABLED = 0
SHADOW_OFFSET_Y_DISABLED = 0
SHADOW_BLUR_DISABLED = 0


### Berechnete Werte
(TODO: Erklärung)

In [ ]:
# all available values for x and y, beginning left to right and top to bottom
av = [
    math.floor(CANVAS_PADDING),
    math.floor(CANVAS_PADDING + ROW_WIDTH),
    math.floor(CANVAS_PADDING + 2 * ROW_WIDTH),
    math.floor(BOARD_SIZE / 2),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + 2 * ROW_WIDTH)),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + ROW_WIDTH)),
    math.floor(BOARD_SIZE - CANVAS_PADDING)
]


### Koordinaten
(TODO: Erklärung)

In [ ]:
# All coordinates. The first tuple is the outer square, the second tuple is the middle square and the third tuple is the inner square. It always starts from top left and then clockwise.
coords = (
    (
        (av[0], av[0]),
        (av[3], av[0]),
        (av[6], av[0]),
        (av[6], av[3]),
        (av[6], av[6]),
        (av[3], av[6]),
        (av[0], av[6]),
        (av[0], av[3])
    ),
    (
        (av[1], av[1]),
        (av[3], av[1]),
        (av[5], av[1]),
        (av[5], av[3]),
        (av[5], av[5]),
        (av[3], av[5]),
        (av[1], av[5]),
        (av[1], av[3])
    ),
    (
        (av[2], av[2]),
        (av[3], av[2]),
        (av[4], av[2]),
        (av[4], av[3]),
        (av[4], av[4]),
        (av[3], av[4]),
        (av[2], av[4]),
        (av[2], av[3])
    )
) 

## Funktionen zum Zeichnen
(TODO: Erklärung)

### drawLine
(TODO: Erklärung)

In [ ]:
# Function to draw a line on a canvas c, from start to end. Start and end are tuples with x and y values.
def drawLine(c, start, end):
    c.move_to(start[0], start[1])
    c.line_to(end[0], end[1])

### toggleShadow
(TODO: Erklärung)

In [ ]:
# Function to enable or disable shadows on a given canvas
def toggleShadow(c, enable):
    c.shadow_color    = SHADOW_COLOR_ENABLED    if enable else SHADOW_COLOR_DISABLED
    c.shadow_offset_x = SHADOW_OFFSET_X_ENABLED if enable else SHADOW_OFFSET_X_DISABLED
    c.shadow_offset_y = SHADOW_OFFSET_Y_ENABLED if enable else SHADOW_OFFSET_Y_DISABLED
    c.shadow_blur     = SHADOW_BLUR_ENABLED     if enable else SHADOW_BLUR_DISABLED

### drawCircle
(TODO: Erklärung)

In [ ]:
# Function to draw a circle on a canvas c with given coords, radius and color
def drawCircle(c, coords, radius, color, strokeColor = None, useShadow = False, selected = False):
    if useShadow:
        toggleShadow(c, True)
    c.fill_style = color
    c.fill_arc(coords[0], coords[1], radius, 0, 2 * math.pi)
    if useShadow:
        toggleShadow(c, False)
    if strokeColor is not None:
        c.line_width = SELECTED_STONE_LINE_WIDTH if selected else DEFAULT_STONE_LINE_WIDTH
        c.strokeColor = strokeColor
        c.stroke_arc(coords[0], coords[1], radius, 0, 2 * math.pi)

### drawStone
(TODO: Erklärung)

In [ ]:
# Function to draw a (player) stone with given coords and for a given player (or the the default )
def drawStone(c, coords, player, selected = False):
    if player == PLAYER_1:
        drawCircle(c, coords, PLAYER_PIECE_RADIUS, PLAYER_1_FILL_COLOR, strokeColor = PLAYER_1_STROKE_COLOR, useShadow = True, selected = selected)
        drawCircle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), PLAYER_1_FILL_COLOR, strokeColor = PLAYER_1_STROKE_COLOR, selected = selected)
    elif player == PLAYER_2:
        drawCircle(c, coords, PLAYER_PIECE_RADIUS, PLAYER_2_FILL_COLOR, strokeColor = PLAYER_2_STROKE_COLOR, useShadow = True, selected = selected)
        drawCircle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), PLAYER_2_FILL_COLOR, strokeColor = PLAYER_2_STROKE_COLOR, selected = selected)
    else:
        drawCircle(c, coords, DEFAULT_PIECE_RADIUS, BOARD_FOREGROUND_COLOR)

### drawText
(TODO: Erklärung)

In [ ]:
def drawText(c, msg, hint = None):
    c.clear()
    c.fill_text(msg, TEXT_X, TEXT_Y, max_width = TEXT_MAX_WIDTH)
    if hint is not None:
        c.fill_text('hint: ' + hint, TEXT_X, TEXT_Y + TEXT_VERTICAL_PADDING, max_width = TEXT_MAX_WIDTH)

### constructSquare
(TODO: Erklärung)

In [ ]:
# Function to construct a square on a canvas c.
def constructSquare(c, square):
    for i in range(4):
        start = i * 2
        end = (i * 2 + 2) if (i * 2 + 2 <= 6) else 0 
        drawLine(c, square[start], square[end])

### constructCrossLines
(TODO: Erklärung)

In [ ]:
# Function to construct the cross lines on a canvas c
def constructCrossLines(c, coords):
    for i in range(4):
        k = i * 2 + 1
        drawLine(c, coords[0][k], coords[2][k])

### setupCanvas
(TODO: Erklärung)

In [ ]:
# Function to setup a canvas, returns a reference to a MultiCanvas
def setupCanvas():
    # level 0: Background and lines
    # level 1: Stones
    # level 2: Text
    canvas = ipycanvas.MultiCanvas(3, width = CANVAS_WIDTH, height = CANVAS_HEIGHT)
    with ipycanvas.hold_canvas(canvas[0]):
        
        canvas[0].fill_style = CANVAS_BACKGROUND_COLOR
        canvas[0].fill_rect(0, 0, CANVAS_WIDTH, CANVAS_HEIGHT)
        
        canvas[0].fill_style = BOARD_BACKGROUND_COLOR
        canvas[0].fill_rect(0, 0, BOARD_SIZE, BOARD_SIZE)

        canvas[0].fill_style = STASH_BACKGROUND_COLOR
        canvas[0].fill_rect(STASH_STARTING_POINT_X, STASH_STARTING_POINT_Y, STASH_WIDTH, STASH_HEIGHT)


        canvas[0].stroke_style = BOARD_FOREGROUND_COLOR
        canvas[0].begin_path()

        for i in range(3):
            constructSquare(canvas[0], coords[i])

        constructCrossLines(canvas[0], coords)
        canvas[0].stroke()
    
    canvas[1].font = TEXT_FONT
    canvas[1].fill_style = TEXT_COLOR
    
    return canvas

### updateGui
(TODO: Erklärung)

In [ ]:
def updateGui(c, state, selectedStone = None):
    with ipycanvas.hold_canvas(c):
        c.clear()
        ((stashP1, stashP2), squares) = state

        # update pieces on the board
        for i in range(len(squares)):
            for j in range(len(squares[i])):
                drawStone(c, coords[i][j], squares[i][j], selected = selectedStone == (i, j))

        # update pieces on the stash

        # player 1

        x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
        y = STASH_STARTING_POINT_Y + CANVAS_PADDING + PLAYER_PIECE_RADIUS

        for i in range(stashP1):
            x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
            drawStone(c, (x, y), PLAYER_1)

        # player 2

        x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
        y = STASH_STARTING_POINT_Y + CANVAS_PADDING + 3 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING

        for i in range(stashP2):
            x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
            drawStone(c, (x, y), PLAYER_2)
    

## Hilfsfunktionen für das Spiel in der GUI
(TODO: Erklärung)

In [ ]:
#%run ./nmm-minimax.ipynb
%run ./nmm-alpha-beta-pruning.ipynb

### getClickedStone
(TODO: Erklärung)

In [ ]:
def getClickedStone(x, y):
    for value in av:
        if value - PLAYER_PIECE_RADIUS <= x <= value + PLAYER_PIECE_RADIUS:
            logger.info(f'value for x = {x} found: {value}')
            x = value
        if value - PLAYER_PIECE_RADIUS <= y <= value + PLAYER_PIECE_RADIUS:
            logger.info(f'value for y = {y} found: {value}')
            y = value

    for i in range(len(coords)):
        for j in range(len(coords[i])):
            if coords[i][j] == (x, y):
                logger.info(f'stone clicked: {(x, y)}')
                return (i, j)
    return None

## Klasse GameState

(TODO: Erklärung)

In [ ]:
class GameState:
    def __init__(self, state = START_STATE, player = PLAYER_1, computer = False):
        self.state = state
        self.player = player
        self.canvas = setupCanvas()
        self.winner = None
        self.resetStateVariables()
        
        self.computer = computer
        
        self.canvas[2].on_mouse_up(self.handleGame)
        
        updateGui(self.canvas[2], self.state)
        self.updateText()
        logger.info('game state initalized')
        
        if self.computer and self.player == PLAYER_2:
            self.nextComputerStep()

### resetStateVariables
(TODO: Erklärung)

In [ ]:
def resetStateVariables(self):
    self.stateTemp = None
    self.millsToPound = 0
    self.selectedStone = None
    self.hint = None
    
GameState.resetStateVariables = resetStateVariables
del resetStateVariables

### handleGame
(TODO: Erklärung)

In [ ]:
def handleGame(self, x, y):
    if self.winner is not None:
        logger.warning('Game has ended!')
        return

    logger.info(f'({x}, {y})')

    phase = playerPhase(self.state, self.player)
    logger.info(f'player phase: {phase}')

    stone = getClickedStone(x, y)

    if stone is None:
        logger.warning('No stone was clicked!')
        if self.selectedStone is not None and self.millsToPound <= 0:
            self.cancelStep()
    elif self.millsToPound > 0:
        self.poundMillInGui(stone)
    elif self.selectedStone is not None:
        self.moveStoneTo(stone)
    elif phase == 1:
        self.playerPhaseOne(stone)
    elif phase == 2 or phase == 3:
        self.selectStone(stone)

    self.checkIfFinished()
    
    self.updateText()
    
    if self.computer and self.player == PLAYER_2:
        self.nextComputerStep()
    
GameState.handleGame = handleGame
del handleGame

### togglePlayer
(TODO: Erklärung)

In [ ]:
def togglePlayer(self):
    self.player = opponent(self.player)
        
GameState.togglePlayer = togglePlayer
del togglePlayer

In [ ]:
def nextComputerStep(self):
    logger.info('next computer step calculating...')
    
    bestValue, bestMove = bestMoveAlphaBeta(self.state, self.player)
    
    logger.info(f'best state calculated: {bestMove}')
    
    self.state = bestMove
    self.togglePlayer()
    self.resetStateVariables()
    updateGui(self.canvas[2], self.state)
    self.checkIfFinished()
    self.updateText()
    
GameState.nextComputerStep = nextComputerStep
del nextComputerStep

### playerPhaseOne
(TODO: Erklärung)

In [ ]:
def playerPhaseOne(self, stone):
    if getPlayerAt(self.state[1], stone) != NO_PLAYER:
        logger.warning(f'{stone} is not free')
        self.hint = f'The slot at {stone} is not free!'
        return

    newState = (removeFromStash(self.state[0], self.player), place(self.state[1], stone, self.player))

    if self.validateNewState(newState):
        logging.info('stone placed')
    else:
        logger.info('NewState not in allAvailableStates, checking for new Mills ...')
        self.checkForNewMills(newState)

        
GameState.playerPhaseOne = playerPhaseOne
del playerPhaseOne

### selectStone
(TODO: Erklärung)

In [ ]:
def selectStone(self, stone):
    if getPlayerAt(self.state[1], stone) != self.player:
        logger.warning(f'{stone} is not the own stone')
        self.hint = 'Plase select your own stone!'
        return
    self.selectedStone = stone
    self.hint = None
    updateGui(self.canvas[2], self.state, selectedStone = self.selectedStone)

    
GameState.selectStone = selectStone
del selectStone

### checkForNewMills
(TODO: Erklärung)

In [ ]:
def checkForNewMills(self, newState):
    oldMills = findMills(self.state[1], self.player)
    newMills = countNewMills(newState[1], oldMills, self.player)

    if newMills > 0:
        
        logger.info(f'new mills: {newMills}')
        
        self.stateTemp = newState
        self.millsToPound = newMills
        self.hint = None
        updateGui(self.canvas[2], self.stateTemp)
    

GameState.checkForNewMills = checkForNewMills
del checkForNewMills

### poundMillInGui
(TODO: Erklärung)

In [ ]:
def poundMillInGui(self, stone):
    if self.millsToPound  <= 0:
        logger.warning('Player has no Mills to pound!')
        return
    if getPlayerAt(self.state[1], stone) != opponent(self.player):
        logger.warning(f'{stone} is not the opponent!')
        self.hint = 'Please select an opponent stone!'
        
        return
    # if the player has only one mill left to pound, it uses the place function and afterwards validates the newState
    if self.millsToPound == 1:
        newState = (self.stateTemp[0], place(self.stateTemp[1], stone, NO_PLAYER))
        if self.validateNewState(newState):
            logger.info('success')
        else:
            logger.warning('Mills could not be pounded! The new state could not be validated by the game logic.')
            self.hint = 'Please do not select an opponent stone that is in a mill!'

        return
    
    # otherwise the gui has to validate the mill manually, as the intermediate step cannot be checked by the game logic
    if stone in getCellsPoundable(self.stateTemp[1], self.player):
        self.stateTemp = (self.stateTemp[0], place(self.stateTemp[1], stone, NO_PLAYER))
        self.millsToPound -= 1
        self.hint = None
        updateGui(self.canvas[2], self.stateTemp)
    else:
        logger.warning('Mills could not be pounded! The new state could not be validated by the gui.')
        self.hint = 'Please do not select an opponent stone that is in a mill!'

GameState.poundMillInGui = poundMillInGui
del poundMillInGui

### validateNewState
(TODO: Erklärung)

In [ ]:
def validateNewState(self, newState):
    allAvailableStates = nextStates(self.state, self.player)
    if newState in allAvailableStates:
        logger.info('SUCCESS')

        self.state = newState
        self.resetStateVariables()
        self.togglePlayer()
        updateGui(self.canvas[2], self.state)
        return True
    else:
        logger.warning('state is not in allAvailableStates!')
        return False

GameState.validateNewState = validateNewState
del validateNewState

### cancelStep
(TODO: Erklärung)

In [ ]:
def cancelStep(self):
    logger.warn('step is canceled.')
    
    self.resetStateVariables()
    updateGui(self.canvas[2], self.state)
    
GameState.cancelStep = cancelStep
del cancelStep

### moveStoneTo
(TODO: Erklärung)

In [ ]:
def moveStoneTo(self, coord):
    if getPlayerAt(self.state[1], coord) != NO_PLAYER:
        logger.warning(f'{coord} is not free')
        self.hint = f'The slot at {coord} is not free!'
        return
    
    canJump = isAllowedToJump(self.state, self.player)
    logger.info(f'moveStoneTo({coord})')

    if canJump or coord in findNeighboringEmptyCells(self.state[1], self.selectedStone):
        newState = (self.state[0], place(self.state[1], self.selectedStone, NO_PLAYER))
        newState = (newState[0], place(newState[1], coord, self.player))

        if self.validateNewState(newState):
            movement = 'jumped' if canJump else 'moved'
            logger.info(f'Stone successfully {movement}!')
        else:
            logger.info('Round not finished, checking for new mills...')
            self.checkForNewMills(newState)
    else:
        logger.warning(f'{coord} is not a (free) neighbor of {self.selectedStone}!')
        self.hint = f'The slot at {coord} is not a (free) neighbor of {self.selectedStone}!'

GameState.moveStoneTo = moveStoneTo
del moveStoneTo

### updateText
(TODO: Erklärung)

In [ ]:
def updateText(self):
    phase = playerPhase(self.state, self.player)
    if self.winner is None:
        message = f'Player {self.player}: '
        if self.computer and self.player == PLAYER_2:
            message = 'Computer\'s turn. Please wait.'
        elif self.millsToPound == 1:
            message += 'Pound your mill.'
        elif self.millsToPound > 1:
            message += f'You have {self.millsToPound} mills left to pound. Please pound your next mill.'
        elif self.selectedStone is not None:
            movement = 'move' if phase == 2 else 'jump'
            message += f'{movement} your selected stone'
        elif phase == 1:
            message += 'Place your stone.'
        elif phase == 2 or phase == 3:
            movement = 'move' if phase == 2 else 'jump'
            message += f'select your stone you want to {movement}'      
    else:
        message = 'The game has ended: '
        if self.winner == NO_PLAYER:
            message += 'Tie'
        else:
            message += f'{self.winner} has won!'
    logging.info(message)
    drawText(self.canvas[1], message, hint = self.hint)

GameState.updateText = updateText
del updateText

### checkIfFinished
(TODO: Erklärung)

In [ ]:
def checkIfFinished(self):
    if finished(self.state, self.player):
        status = utility(self.state, self.player)
        if status == 0:
            self.winner = NO_PLAYER
        else:
            self.winner = self.player if status == 1 else opponent(self.player)

GameState.checkIfFinished = checkIfFinished
del checkIfFinished

In [ ]:
gameState = GameState(computer = True)

In [ ]:
gameState.canvas